### 진행방향
집계구 코드를 불러와서 지도위에 ploting  
데이터를 병합하는 방안을 고려해볼것
예상) 업종별, 연령별 - 이용건수 및 이용금액

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium 
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from glob import glob
import os
pbar = ProgressBar()
pbar.register()

from tqdm import tqdm_notebook as tqdm
import geopandas as gpd

#한글깨짐 방지
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

### data load
통합파일 저장시, index = False조건을 안해놈.. 나중 수정..

In [ ]:
s_2017 = pd.read_csv('E:/python/data/신한카드데이터/내국인 성별연령별_2017년_신한_종합.csv').iloc[:,1:]
gu = gpd.read_file('E:/python/data/1. 공통참조파일/1. 지역경계 shp파일/통계청/2016년/서울시(집계구)_EPSG_5179.shp',
                  encoding = 'cp949')

#### geopandas 시각화 확인

In [ ]:
gu.plot(column = 'ADM_NM')

### 2017년 월별 시간대별로 군집
* 특히 t3시간대(12~17:59)로 확인

In [ ]:
grouped_year_time = s_2017.groupby(['BLOCK_ID','TS_YM','TM'])['USECT_CORR','AMT_CORR'].sum().reset_index()

### 집계구 광진구만 발라내기

In [ ]:
gwang_code = pd.read_csv('./data/광진구_행정동코드.csv',encoding = 'cp949')

In [ ]:
gwangjin_dong = gwang_code['행정동'].values

In [ ]:
gwangjin_gu = gu[gu.ADM_NM.isin(gwangjin_dong)]

In [ ]:
# 각 집계구의 중심점 확인
gwangjin_gu['center_point'] = gwangjin_gu.geometry.apply(lambda x : x.centroid)

### 데이터결합

In [ ]:
#타입이 다르므로 통일
s_2017.BLOCK_ID = s_2017.BLOCK_ID.astype(str)
s_2017.M_DONG_CD = s_2017.M_DONG_CD.astype(str)

In [ ]:
merged = gwangjin_gu.merge(grouped_year_time, how = 'outer',left_on = 'TOT_REG_CD',right_on = 'BLOCK_ID')

#### 각 집계구의 중심점 

In [ ]:
merged['center_x'] = merged.center_point.apply(lambda point : point.x)
merged['center_y'] = merged.center_point.apply(lambda point : point.y)

### sample시각화
#샘플로 17년 1월 t3시간의 값만 가져옴

In [ ]:
sample = merged.loc[(merged.TS_YM == 201701)&(merged.TM == 't3')]

### 지도시각화
* 누적값으로 구하다보니 집중되는 지역에 편차가 심함.  
-> 비율또한 의미 없음. 상위 5개 구간을 제외하고 그림 그려보기 

In [ ]:
#수치를 다가져가는 동네를 제외하고 작업해야할것으로 추정
fig,ax = plt.subplots(1,1,figsize = (15,15))
sample.plot(column = 'AMT_CORR',ax = ax, legend = True,edgecolor = 'black',cmap = 'OrRd')
plt.scatter(x = 'center_x',y = 'center_y',data= sample,s = 10, color = 'b')
plt.title('집계구별 t3(12~17:59)사이 이용금액',size= 15)
plt.axis('off')
plt.show()

In [ ]:
#상위 5개 지역 결측처리후 진행
sample1 = sample.sort_values('AMT_CORR',ascending = False)
sample1.iloc[:6]['AMT_CORR'] = np.nan
fig,ax = plt.subplots(1,1,figsize = (15,15))
sample1.plot(column = 'AMT_CORR',ax = ax, legend = True,
                       edgecolor = 'black',cmap = 'OrRd',
                      missing_kwds= {
                          "color": "lightgray",
                          "edgecolor": "red",
                          "hatch": "///",
                      })
plt.scatter(x = 'center_x',y = 'center_y',data= sample,s = 10, color = 'b')
plt.title('집계구별 t3(12~17:59)사이 이용금액',size= 15)
# plt.axis('off')
plt.show()

### 특정 지역만 확인
상권분석 서비스의 상권데이터로 특정위치 확인

In [ ]:
sangkwon = gpd.read_file('E:/python/data/상권분석서비스/서울시_우리마을가게_상권분석서비스(상권영역)/TBGIS_TRDAR_RELM.shp')
sangkwon = sangkwon.to_crs(5179) #좌표값변환
gwangjin_sangkwon = sangkwon[sangkwon.SIGNGU_CD == '11215']

In [ ]:
#주변 주요 상권명
near_sangkwon = ['어린이대공원역','능동로19길','광나루로24길',
 '광나루로36길','광나루로37길','광나루로 40길','광나루로 26길',
 '자양로50길','천호대교129길','자양로45길','자양로53길','천호대교123길',
 '능동로34길','서울 광진구 군자역','용마산로3길','천호대교3길']

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (15,15))
sample.plot(ax = ax,edgecolor = 'black',color = 'w')
gwangjin_sangkwon.plot(column = 'TRDAR_CD_N', ax = ax, legend = True)
plt.scatter(x = 'center_x',y = 'center_y',data= sample,s = 10, color = 'red')
plt.title('상권지역위의 포인트',size= 15)
plt.axis('off')
plt.show()

In [ ]:
#인근상권만
near_park = gwangjin_sangkwon[gwangjin_sangkwon.TRDAR_CD_N.isin(near_sangkwon)]
fig,ax = plt.subplots(1,1,figsize = (15,15))
sample.plot(ax = ax,edgecolor = 'black',color = 'w')
near_park.plot(column = 'TRDAR_CD_N', ax = ax, legend = True, cmap = 'tab20')
plt.scatter(x = 'center_x',y = 'center_y',data= sample,s = 10, color = 'red')
plt.title('상권지역위의 포인트',size= 15)
plt.axhline(y = 1951100) #범위확인
plt.axhline(y = 1949300) #범위확인
plt.axvline(x = 962000)
plt.axvline(x = 964000)
plt.ylim((1949300,1951100))
plt.xlim((962000,964000))
# plt.axis('off')
plt.show()

In [ ]:
# 범위확인
ymin,ymax = 1949300, 1951100
xmin,xmax = 962000,964100

In [ ]:
#인근상권만
near_park = gwangjin_sangkwon[gwangjin_sangkwon.TRDAR_CD_N.isin(near_sangkwon)]
fig,ax = plt.subplots(1,1,figsize = (15,15))
sample.plot(column = 'AMT_CORR',ax = ax,edgecolor = 'black',cmap = 'OrRd')
near_park.plot(column = 'TRDAR_CD_N', ax = ax, legend = True, cmap = 'tab20',alpha = 0.2)
plt.scatter(x = 'center_x',y = 'center_y',data= sample,s = 10, color = 'red')
plt.title('축소범위체크',size= 15)

plt.axhline(y = ymin) 
plt.axhline(y = ymax) 
plt.axvline(x = xmin)
plt.axvline(x = xmax)

# plt.axis('off')
plt.show()

In [ ]:
sample[sample.BLOCK_ID.isna()]

In [ ]:
#인근상권만
near_park = gwangjin_sangkwon[gwangjin_sangkwon.TRDAR_CD_N.isin(near_sangkwon)]
fig,ax = plt.subplots(1,1,figsize = (15,15))
sample.plot(column = 'AMT_CORR',ax = ax,edgecolor = 'black',cmap = 'OrRd')
near_park.plot(column = 'TRDAR_CD_N', ax = ax, legend = True, cmap = 'tab20',alpha = 0.2)
plt.scatter(x = 'center_x',y = 'center_y',data= sample,s = 10, color = 'red')
plt.title('지역축소',size= 15)

plt.ylim((ymin,ymax))
plt.xlim((xmin,xmax))
# plt.axis('off')
plt.show()

### 축소한 범위내의 상권만 분석대상으로

### 클러스터링 다시

In [ ]:
## merge에 na있어서 다시 
merged = merged[~(merged.BLOCK_ID.isna())]
target = n_merged[(n_merged.center_x >= xmin)&(n_merged.center_x <= xmax)]
target = n_target[(n_target.center_y >= ymin)&(n_target.center_y <= ymax)]

In [ ]:
#인근상권만
near_park = gwangjin_sangkwon[gwangjin_sangkwon.TRDAR_CD_N.isin(near_sangkwon)]
fig,ax = plt.subplots(1,1,figsize = (15,15))
target.plot(column = 'AMT_CORR',ax = ax,edgecolor = 'black',cmap = 'OrRd')
near_park.plot(column = 'TRDAR_CD_N', ax = ax, legend = True, cmap = 'tab20',alpha = 0.5)
plt.scatter(x = 'center_x',y = 'center_y',data= target,s = 10, color = 'red')
plt.title('지역축소',size= 15)

# plt.ylim((ymin,ymax))
# plt.xlim((xmin,xmax))
# plt.axis('off')
plt.show()

### kmeans로 클러스터링

In [ ]:
from sklearn.cluster import KMeans

center_points = target[['center_x','center_y']].drop_duplicates()
kmeans = KMeans(n_clusters = 10).fit(center_points)
center_points['labels'] = kmeans.labels_

In [ ]:
#인근상권만
near_park = gwangjin_sangkwon[gwangjin_sangkwon.TRDAR_CD_N.isin(near_sangkwon)]
fig,ax = plt.subplots(1,1,figsize = (15,15))
target.plot(column = 'AMT_CORR',ax = ax,edgecolor = 'black',cmap = 'OrRd')
near_park.plot(column = 'TRDAR_CD_N', ax = ax,cmap = 'tab20',alpha = 0.5)
plt.title('클러스터링',size= 15)
sns.scatterplot(x = 'center_x',y = 'center_y',hue= 'labels',s = 80, 
                data = center_points, palette = 'Accent_r',legend = 'full')
plt.yticks(fontsize = 15)
plt.xticks(fontsize = 15)
plt.show()

### 특정 좌표 제외작업

In [ ]:
#건대주변 상권제외작업

In [ ]:
#인근상권만
near_park = gwangjin_sangkwon[gwangjin_sangkwon.TRDAR_CD_N.isin(near_sangkwon)]
fig,ax = plt.subplots(1,1,figsize = (15,15))
target.plot(column = 'AMT_CORR',ax = ax,edgecolor = 'black',cmap = 'OrRd')
# near_park.plot(column = 'TRDAR_CD_N', ax = ax,cmap = 'tab20',alpha = 0.5)
plt.title('건대상권제외',size= 15)
sns.scatterplot(x = 'center_x',y = 'center_y',hue= 'labels',s = 80, 
                data = center_points, palette = 'Accent_r',legend = 'full')
plt.axhline(y = 1949600)
plt.axvline(x = 962250)
plt.yticks(fontsize = 15)
plt.xticks(fontsize = 15)
plt.show()

In [ ]:
gun_x, gun_y = 962250, 1949600
target_g = target[~((target.center_x <= gun_x)&((merged.center_y <= gun_y)))]

In [ ]:
center_points_g = center_points[~((center_points.center_x <= gun_x)&((center_points.center_y <= gun_y)))]

In [ ]:
#대공원 label은 별로도표기 99로 해둠
center_points_g.loc[((center_points_g.center_x >= 962900)&(center_points_g.center_x <= 963100)&
               (center_points_g.center_y >= 1950000)&(center_points_g.center_y <= 1950250)),'labels'] = 10

In [ ]:
target_block = target[['BLOCK_ID','center_x','center_y']].drop_duplicates()
center_points_g = pd.merge(center_points_g,target_block, on = ['center_x','center_y'],how = 'left')
center_points_save = center_points_g[['BLOCK_ID','labels']]

In [ ]:
# # #클러스터 저장
# center_points_save.to_csv('./data/신한카드데이터/블럭별 클러스터라벨.csv',index= False)

In [ ]:
#인근상권만
near_park = gwangjin_sangkwon[gwangjin_sangkwon.TRDAR_CD_N.isin(near_sangkwon)]
fig,ax = plt.subplots(1,1,figsize = (15,15))
target_g.plot(column = 'AMT_CORR',ax = ax,edgecolor = 'black',cmap = 'OrRd')
# near_park.plot(column = 'TRDAR_CD_N', ax = ax,cmap = 'tab20',alpha = 0.5)
plt.title('지역축소',size= 15)
sns.scatterplot(x = 'center_x',y = 'center_y',hue= 'labels',s = 80, 
                data = center_points_g, palette = 'gist_ncar',legend = 'full')
plt.axis('off')
plt.show()

In [ ]:
target_g.BLOCK_ID.nunique()

### 각 클러스터병합

In [ ]:
from shapely.ops import cascaded_union

In [ ]:

cluster_name = {}
cluster_name[0] = "구의문"
cluster_name[1] = "세종대 뒤편"
cluster_name[2] = "남문"
cluster_name[3] = "후문"
cluster_name[4] = "어린이대공원정문"
cluster_name[5] = "군자역 뒤편"
cluster_name[6] = "군자역"
cluster_name[7] = "구의문 뒤편"
cluster_name[8] = "후문뒤편"
cluster_name[9] = "서문,북문"
cluster_name[10] = "어린이대공원"

cluster_name_df = pd.DataFrame({'labels': list(cluster_name.keys()), 'names': list(cluster_name.values())})

In [ ]:
cluster_name = {}
cluster_name[0] = "군자역 뒤편"
cluster_name[1] = "어린이대공원정문"
cluster_name[2] = "구의문 뒤편"
cluster_name[3] = "세종대 뒤편" 
cluster_name[4] = "후문"
cluster_name[5] = "후문뒤편"
cluster_name[6] = "서문,북문"
cluster_name[7] = "남문"
cluster_name[8] = "구의문"
cluster_name[9] = "군자역"
cluster_name[10] = "어린이대공원"

cluster_name_df = pd.DataFrame({'labels': list(cluster_name.keys()), 'names': list(cluster_name.values())})

In [ ]:
union_polygon = {}
for label in near_data.labels.unique():
    union_polygon[label] = cascaded_union(near_data[near_data.labels == label].geometry)

geo_df = pd.DataFrame({'labels': list(union_polygon.keys()), 'geometry': list(union_polygon.values())})

In [ ]:
geo_df = gpd.GeoDataFrame(pd.merge(cluster_name_df,geo_df))

In [ ]:
geo_df.plot(column = 'names',edgecolor = 'black',cmap = 'tab20', figsize = (15,15),legend = True)

In [ ]:
#저장
geo_df.to_file('./data/신한카드데이터/라벨별_명칭_geo값.shp',encoding = 'cp949')

In [ ]:
#주변상권데이터와 라벨링 결합
near_data = pd.merge(merged,center_points_g,how = 'left', on = ['center_x','center_y'])
near_data = near_data.dropna()

In [ ]:
near_data = pd.merge(near_data,cluster_name_df)

In [ ]:
near_data_gb = near_data.groupby(['names','TS_YM'])['USECT_CORR','AMT_CORR'].sum().reset_index()

In [ ]:
plt.figure(figsize = (10,10))
sns.lineplot(x = 'TS_YM', y= 'AMT_CORR', hue = 'names', data = near_data_gb, palette= 'tab20')
plt.legend(loc = 'center left', bbox_to_anchor = (1,0.5),fontsize = 12)
plt.title('주변상권의 월간추세',fontsize= 15)
plt.yticks(fontsize = 15)
plt.xticks(fontsize = 15)
plt.tight_layout()
plt.savefig('./output/신한카드/주변상권의 월간추세(2017).png')
plt.show()

### 지도시각화

In [ ]:
near_data_gb = near_data.groupby(['labels','names'])['USECT_CORR','AMT_CORR'].sum().reset_index()

In [ ]:
near_data_gb = pd.merge(near_data_gb,geo_df)

In [ ]:
near_data_gpd = gpd.GeoDataFrame(near_data_gb)

In [ ]:
#2017 군자역부근과 정문부근이 매출이 높음 
fig,ax = plt.subplots(1,1,figsize = (15,15))
near_data_gpd.plot(column = 'AMT_CORR',ax = ax,edgecolor = 'black',cmap = 'OrRd')